# Background on Taproot

## Where does "taproot" come from?

Taproot is a series of bitcoin BIPS (bitcoin improvement proposals) that were merged into bitcoin-core in 2021.

It was most recent bitcoin soft-fork and it went live in November 2021.

The BIPS can be found on github. They are as follows:

- BIP340, Schnorr Signatures https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki
- BIP341, Taproot Segwit v1 https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- BIP342, Validating Taproot Spends https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki
- BIP350, Bech32m https://github.com/bitcoin/bips/blob/master/bip-0350.mediawiki

## What did "taproot" change?

Taproot added and modified many things in bitcoin. Including

- A new signature algorithm (Schnorr)
- A new version of Script opcodes (Segwit v1)
- A new bitcoin address checksum (bech32m)
- Removed some opcodes we know and love (OP_CHECKMULTISIG)
- Added a new opcode (OP_CHECKSIGADD)
- Added a new scripthash algorithm

# Finding Taproot in the Wild

Let's go to mempool.space and find some taproot addresses in the wild!



## How to spot a taproot address

The bech32 address has a `p` immediately following the `1`

Find a taproot address on mempool dot space.

What's the "Type" for this address?


Copy paste the "ScriptPubKey (HEX)" for the address you found into the pyblock below.

In [1]:
scriptpubkey = "51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953"

In [3]:
print(scriptpubkey)

51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953


In [9]:
len(scriptpubkey) // 2

34

This is a series of opcodes. If you pass this to `bitcoin-cli decodescript` what does it print back as the asm?

In [31]:
!bitcoin-cli -regtest decodescript 51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953

{
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}


In [33]:
output = {
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}

In [34]:
print(output['asm'])

1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953


In [36]:
output['asm'].split(" ")

['1', '3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953']

In [37]:
len(output['asm'].split(" ")[1]) // 2

32

In [38]:
print(output['type'])

witness_v1_taproot


In [131]:
x_bytes = bytes.fromhex(output['asm'].split(' ')[1])
print('hex:\t\t', x_bytes.hex())

x_val = int.from_bytes(x_bytes, 'big')
print('int val:\t', x_val)

hex:		 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953
int val:	 25792158117016797743484614046835601668965634494428223733945610235101691103571


## Finding the secp256k1 pubkey from a P2TR address

Now that we have the x-only pubkey, let's figure out what point that is on the secp256k1 curve.

We can do that using `lift_x_pubkey`. This takes an integer and returns a point.

In my case, I'll use `coincurve`, a python library for expressing + doing math with public keys.

In [40]:
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
def lift_x(x_value):
    assert x_value < p
    # find y squared (y2 = x^3 + 7)
    y2 = (x_value ** 3 + 7) % p
    # find the square root of y
    y = pow(y2, (p+1)//4, p)
    assert (y ** 2 % p) == y2
    # if y is odd, find even y
    if y % 2 != 0:
        y = p - y
    return coincurve.PublicKey.from_point(x_value, y)

In [41]:
import coincurve

point = lift_x(x_val)
print(point.format().hex())
print(point.point())

023905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953
(25792158117016797743484614046835601668965634494428223733945610235101691103571, 4191453718700008539754125939901750860298491226938149986069530358203844164198)


## Let's Get Familiar With The Tools!

In [61]:
from codes import parse_tx_bytes

tx = "020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000"

In [62]:
print(parse_tx_bytes(tx))

AssertionError: 

In [63]:
from pprint import pprint

In [64]:
pprint(parse_tx_bytes(tx))

AssertionError: 

First things first, let's update the `parse_tx_bytes` method to be able to handle segwit (any version) transactions.

In [65]:
from codes import parse_compact_size, parse_input_bytes, parse_output_bytes

def parse_tx_bytes_mine(tx_hex):
  tx_bytes = bytes.fromhex(tx_hex)

  tx = {}
  ptr = 0
  tx['version'] = tx_bytes[0:4]
  ptr += 4

  if tx_bytes[ptr] == 0x00:
    assert tx_bytes[ptr+1] == 0x01
    tx['marker_flag'] = bytes([0x00, 0x01])
    ptr += 2

  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size 
  tx['inputs'] = []
  for _ in range(0, count):
    inputx, size = parse_input_bytes(tx_bytes[ptr:])
    ptr += size
    tx['inputs'].append(inputx)
 
  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size
  tx['outputs'] = []
  for _ in range(0, count):
    outputx, size = parse_output_bytes(tx_bytes[ptr:])
    ptr += size
    tx['outputs'].append(outputx)

  if 'marker_flag' in tx:
    tx['witnesses'] = []
    for _ in range(0, len(tx['inputs'])):
        witness, size = parse_witness_bytes(tx_bytes[ptr:])
        ptr += size
        tx['witnesses'].append(witness)

  tx['locktime'] = tx_bytes[ptr:]
  return tx

In [66]:
tx_bytes = bytes.fromhex(tx)
ptr = 4
print(tx_bytes.hex())
print(tx_bytes[ptr:].hex())

020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000
0001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000


In [107]:
pprint(parse_tx_bytes_mine(tx))

{'inputs': [{'scriptSig': b'',
             'sequence': b'\xff\xff\xff\xff',
             'txid': b'<)!\x1e.k\x9aYO(\xfd\x9e\x8be\r\xd2\xf6Np$'
                     b'\xd3\x98\xfb\xc4\xc9\x8f;\x88\x00\xce\xf15',
             'vout': b'\x00\x00\x00\x00'}],
 'locktime': b'\x00\x00\x00\x00',
 'marker_flag': b'\x00\x01',
 'outputs': [{'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'\xeb\xa6\r\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O

In [68]:
def parse_witness_bytes(data):
    count, size = parse_compact_size(data)
    ptr = size
    witnesses = []
    for _ in range(0, count):
        witlen, size = parse_compact_size(data[ptr:])
        ptr += size
        witnesses.append(data[ptr:ptr+witlen])
        ptr += witlen

    return witnesses, ptr

In [69]:
txdata = parse_tx_bytes_mine(tx)

In [133]:
assert len(txdata['witnesses'][0]) == 1

In [71]:
print(txdata['witnesses'][0][0].hex())

bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de


In [215]:
sig = txdata['witnesses'][0][0]

In [73]:
print(sig.hex())

bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de


In [76]:
print(len(sig))

64


## Validating a Signature

What message did this signature sign?

Finding the "sighash" of that message. 

Take the bitcoin transaction that the signature was embedded into. From this transaction, we'll create a hash.

That hash of the bitcoin transaction "message digest" which is the hash that the signature "signed".

In [74]:
print(tx)

020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000


In [75]:
pprint(txdata)

{'inputs': [{'scriptSig': b'',
             'sequence': b'\xff\xff\xff\xff',
             'txid': b'<)!\x1e.k\x9aYO(\xfd\x9e\x8be\r\xd2\xf6Np$'
                     b'\xd3\x98\xfb\xc4\xc9\x8f;\x88\x00\xce\xf15',
             'vout': b'\x00\x00\x00\x00'}],
 'locktime': b'\x00\x00\x00\x00',
 'marker_flag': b'\x00\x01',
 'outputs': [{'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'\xeb\xa6\r\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O

## Find the SigMsg for a transaction

Here we use the BIP341 Signature Validation Rules (https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#user-content-Common_signature_message) Common signature message definition to calculate the SIGHASH_DEFAULT for a given transaction.

We'll use this to compute the sighash for this input, which we need to validate the signature.

In [104]:
from hashlib import sha256
from codes import size_compact_size

def sigmsg_default(tx_hex, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag):
    txdata = parse_tx_bytes_mine(tx_hex)
    
    result = b''
    result += bytes([0x00]) # sighash_flag
    result += txdata['version']
    result += txdata['locktime']

    all_input_outpoints = b''
    for inp in txdata['inputs']:
        # txid || vout
        all_input_outpoints += inp['txid'] + inp['vout']    
    sha_prevouts = sha256(all_input_outpoints).digest()
    result += sha_prevouts
    
    sha_amounts = sha256(b''.join(amounts_bytes)).digest()        
    result += sha_amounts
    
    spks = b''
    for spk in scriptpubkeys_bytes:
        spks += size_compact_size(len(spk)) + spk
    sha_scriptpubkeys = sha256(spks).digest()
    result += sha_scriptpubkeys
    
    all_sequences = [ i['sequence'] for i in txdata['inputs'] ]
    sha_sequences = sha256(b''.join(all_sequences)).digest()
    result += sha_sequences
    
    all_outputs = b''
    for o in txdata['outputs']:
        # amount || compact_size(scriptpubkey) || scriptpubkey
        all_outputs += o['amount']
        all_outputs += size_compact_size(len(o['scriptPubKey']))
        all_outputs += o['scriptPubKey']
    sha_outputs = sha256(all_outputs).digest()
    result += sha_outputs
    
    # data about this input
    annex_present = 1 if annex_bytes else 0
    spend_type = 2 * ext_flag + annex_present
    result += (spend_type).to_bytes(1, 'little')
    result += (input_index).to_bytes(4, 'little')
    
    if (annex_bytes):
        size_bytes = size_compact_size(len(annex_bytes))
        sha_annex = sha256(size_bytes + annex_bytes).digest()
        result += sha_annex
    
    # data about this output
    # noop for SIGHASH_DEFAULT
    
    assert len(result) <= 206
    return result

In [98]:
spent_from_tx = parse_tx_bytes_mine("02000000000101d79ca7455775b3fc7bf65eae536faa22e6248b863247cee30765fabb4fafadb80100000000ffffffff02dcab0e00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953944be70400000000225120dc2e293b7717ece7898fefc8fd7f6da13e8fb56efac0bc3bd75183304da6ad1701409dab8ddd9c703d1acb6736bc5260b23e64999cb130b9b3ca6c2e69aa084073302b66f0ecb5b4a191618951c022da2c429ac3638613c43bbb88a761634c2cb85300000000")

print(spent_from_tx['outputs'][0])

{'amount': b'\xdc\xab\x0e\x00\x00\x00\x00\x00', 'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t\xc9S'}


In [105]:
amounts_bytes = [spent_from_tx['outputs'][0]['amount']]
print("amounts:", [x.hex() for x in amounts_bytes])

scriptpubkeys_bytes = [spent_from_tx['outputs'][0]['scriptPubKey']]
print("scriptpubkeys:", [x.hex() for x in scriptpubkeys_bytes], "\n")

ext_flag = 0
annex_bytes = None
input_index = 0

sigmsg = sigmsg_default(tx, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag)

print("sigmsg:", sigmsg.hex())

amounts: ['dcab0e0000000000']
scriptpubkeys: ['51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953'] 

sigmsg: 000200000000000000002b9b64a25d11192d251c2e088e7cbc65889963ff67244102b5fc7cb1b4feb51c4eb25d1adc8a44ed6dd54bca7d3d790a7b9e66ef9a8416836882124ad71bb507d41f17121f975dfae4cb8a052b3e34576e1c272bf7e17dfe484be8d1f96317ad95131bc0b799c0b1af477fb14fcf26a6a9f76079e48bf090acb7e8367bfd0e5cdaded8d0fdd6bcb42703ac71156c5d658a47acaa0b90924087e48059d5f1c30000000000


## Computing the Sighash

Now that we have the sighash message, we can calculate the actual "sighash" for the signature.

The `sighash` for Taproot key path spending signature valdiation is as follows:

```
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
```

We'll need to implement the `tag_hash` function, then pass our `sigmsg` into this to find the `sighash`.

In [126]:
from hashlib import sha256

def tag_hash(tag, data_bytes):
    taghash = sha256(tag).digest()
    return sha256(taghash + taghash + data_bytes).digest()

In [127]:
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
print(sighash.hex())

c09a12ac690db9c044c0df972aec9f574cf12677b80b1b82e910bf9e16f3b115


## Verifying the Sighash

Now that we've got the `sighash` we're finally ready to verify the signature.

For this, we'll need the `verify` function, which is defined in BIP340.

In [134]:
# there are three inputs to the verify function: pubkey, message, signature
pk = x_val
m = sighash
sig = sig

print(x_val)

25792158117016797743484614046835601668965634494428223733945610235101691103571


In [219]:
from codes import n

def verify(x_val, m, sig):
    P = lift_x(x_val)
    assert len(sig) == 64
    r = int.from_bytes(sig[:32], 'big')
    if r >= p:
        return False
    s = int.from_bytes(sig[32:], 'big')
    if s >= n:
        return False
    
    P_bytes = (x_val).to_bytes(32, 'big')
    challenge_hash = tag_hash(b'BIP0340/challenge', sig[:32] + P_bytes + m)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # e * P
    eP = P.multiply(e.to_bytes(32, 'big'))
    # s * G
    sG = coincurve.PrivateKey.from_int(s).public_key
    neg_eP = invert_point(eP)
    
    # R = sG - eP = sG + -eP
    R = coincurve.PublicKey.combine_keys([sG, neg_eP])
    
    if not has_even_y(R):
        return False
    
    return R.point()[0] == r

In [209]:
def invert_point(pt):
    x, y = pt.point()
    yneg = p - y
    return coincurve.PublicKey.from_point(x, yneg)

In [210]:
def has_even_y(pt):
    return pt.point()[1] % 2 == 0

In [220]:
sig = txdata['witnesses'][0][0]
verify(x_val, m, sig)

True

## Signing a message

Now that we've been able to successfully verify a message, let's finish implementing the second method for Schnorr: the Sign method.

This takes a private key and a message and returns a valid signature.

```
sig = Sign(privkey, msg)
```

In [224]:
def scalar_to_pubkey(scalar_val):
    assert scalar_val != 0 and scalar_val < n
    P = coincurve.PrivateKey.from_int(scalar_val).public_key
    result = scalar_val if has_even_y(P) else n - scalar_val
    
    return result, P


def sign_msg(privkey_int, msg, nonce_int):
    d, P = scalar_to_pubkey(privkey_int)
    k, R = scalar_to_pubkey(nonce_int)
    
    bytes_R = (R.point()[0].to_bytes(32, 'big'))
    bytes_P = (P.point()[0].to_bytes(32, 'big'))
    challenge_hash = tag_hash(b'BIP0340/challenge', bytes_R + bytes_P + msg)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # ~~~BEGIN SCHNORR PART~~~
    s = (k + e * d) % n
    # ~~~ END SCHNORR PART ~~~
    
    sig = bytes_R + s.to_bytes(32, 'big')
    assert verify(P.point()[0], m, sig)
    return sig

In [225]:
privkey = 10
nonce = 5
sig = sign_msg(privkey, m, nonce)

print(sig.hex())

2f8bde4d1a07209355b4a7250a5c5128e88b84bddc619ab7cba8d569b240efe4de48d2d7d8f794b7e45e4701c5385b94d87543365be474d9941f15d4ecf211a2
